# Principal Component Analysis

PCA é um método de redução de dimensionalidade que transforma pontos de um espaço N-dimensional (amostras com N atributos) em um OUTRO espaço de dimensão K < N (nova base de dados onde as amostras possuem K atributos, provevelmente sem interpretação trivial).

Este método aplica uma transformação linear nos dados de forma que 'boa parte' da informação seja preserveda (sempre haverá perda de informação) por meio da combinação de atributos. A ideia por traz desse método é aplicar uma transformação linear de tal forma que a variância dos dados transformados seja maximizada e por fim selecionar quais dos novos 'eixos' apresentam a mais variância. Assim, além de poder reduzir a dimensionalidade, é possível melhorar a separação entre as populações (se os dados forem linearmente separaveis).

Este é um método não superviosionado porque não requer informação _a priori_ sobre a rotulação dos dados.

![title](pca.gif)

# Linear Discriminant Analysis

LDA é um método, que assim como o PCA, aplica uma transformação linear aos dados para melhorar a separabilidade. No entanto, ao contrario do PCA, este método é superviosionado e demanda a classe (ou rótulo) a qual cada amostra pertence porque, ao invez de simplemente maximizar a variância total, este método busca maximizar a distância entre as médias das classes ao mesmo tempo que minimiza a dispersão intra classes.

Como consequencia da rotulação, além de servir como redutor de dimesionalidade, este método também pode ser utilizado como classificador ao estabelecer divisas lineares entres as classes.

![title](lda.png)

In [ ]:
import random
import numpy as np
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
n = 500
populations = {
    'a':{
        'mean': [0, 0, 0],
        'std': [2, 2, 2],
        'n': n
    },
    'b':{
        'mean': [20, 0, 0],
        'std': [2, 1, 1],
        'n': n
    },
    'b\'':{
        'mean': [-20, 0, 0],
        'std': [2, 1, 1],
        'n': n
    },
    'c':{
        'mean': [0, 20, 0],
        'std': [1, 2, 1],
        'n': n
    },
    'c\'':{
        'mean': [0, -20, 0],
        'std': [1, 2, 1],
        'n': n
    },
    'd':{
        'mean': [0, 0, 20],
        'std': [1, 1, 2],
        'n': n
    },
    'd\'':{
        'mean': [0, 0, -20],
        'std': [1, 1, 2],
        'n': n
    }
}

for name in populations:
    population = populations[name]
    samples = []
    for n in range(population['n']):
        sample = []
        for mean, std in zip(population['mean'], population['std']):
            sample.append(
                random.gauss(mean, std)
            )
        samples.append(sample)
        
    population['data'] = np.array(samples)

In [ ]:
fig = go.Figure(layout=dict(height=800))
for name in populations:
    population = populations[name]
    fig.add_trace(
        go.Scatter3d(
            x = population['data'][:,0],
            y = population['data'][:,1],
            z = population['data'][:,2],
            name=name,
            mode = 'markers',
            marker=dict(
                size=3
            )
        )
    )
fig

In [ ]:
fig = go.Figure(layout=dict(title='XY'))
for name in populations:
    population = populations[name]
    components = [0, 1]
    x = population['data'][:,components[0]]
    y = population['data'][:,components[1]]
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode = 'markers',
            name=name,
                
        )
    )
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=population['mean'][components[0]] - population['std'][components[0]], y0=population['mean'][components[1]] - population['std'][components[1]],
        x1=population['mean'][components[0]] + population['std'][components[0]], y1=population['mean'][components[1]] + population['std'][components[1]],
    )

fig.show()

fig = go.Figure(layout=dict(title='XZ'))
for name in populations:
    population = populations[name]
    components = [0, 2]
    x = population['data'][:,components[0]]
    y = population['data'][:,components[1]]
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode = 'markers',
            name=name,
                
        )
    )
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=population['mean'][components[0]] - population['std'][components[0]], y0=population['mean'][components[1]] - population['std'][components[1]],
        x1=population['mean'][components[0]] + population['std'][components[0]], y1=population['mean'][components[1]] + population['std'][components[1]],
    )

fig.show()

fig = go.Figure(layout=dict(title='YZ'))
for name in populations:
    population = populations[name]
    components = [1, 2]
    x = population['data'][:,components[0]]
    y = population['data'][:,components[1]]
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode = 'markers',
            name=name,
                
        )
    )
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=population['mean'][components[0]] - population['std'][components[0]], y0=population['mean'][components[1]] - population['std'][components[1]],
        x1=population['mean'][components[0]] + population['std'][components[0]], y1=population['mean'][components[1]] + population['std'][components[1]],
    )

fig.show()

In [ ]:
# agregando todos os valores em um único conjunto
all_data = np.concatenate([populations[name]['data'] for name in populations])
all_labels = np.concatenate([[name] * len(populations[name]['data']) for name in populations])
# processando as componentes principais
pca = PCA(n_components=2).fit(all_data)
lda = LinearDiscriminantAnalysis(n_components=2).fit(all_data, all_labels)

for name in populations:
    population = populations[name]
    
    population['pca'] = pca.transform(population['data'])
    population['lda'] = lda.transform(population['data'])
    
    population['pca_mean'] = pca.transform([population['mean']])[0]
    population['lda_mean'] = lda.transform([population['mean']])[0]
    
    population['pca_std'] = pca.transform([population['std']])[0]
    population['lda_std'] = lda.transform([population['std']])[0]
    
    

In [ ]:
fig = go.Figure(layout=dict(title='PCA', height=800))
for name in populations:
    population = populations[name]
    components = [0, 1]
    x = population['pca'][:,components[0]]
    y = population['pca'][:,components[1]]
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode = 'markers',
            name=name,
                
        )
    )
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=population['pca_mean'][components[0]] - population['pca_std'][components[0]], y0=population['pca_mean'][components[1]] - population['pca_std'][components[1]],
        x1=population['pca_mean'][components[0]] + population['pca_std'][components[0]], y1=population['pca_mean'][components[1]] + population['pca_std'][components[1]],
    )

fig

In [ ]:
fig = go.Figure(layout=dict(title='LDA', height=800))
for name in populations:
    population = populations[name]
    components = [0, 1]
    x = population['lda'][:,components[0]]
    y = population['lda'][:,components[1]]
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode = 'markers',
            name=name,
                
        )
    )
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=population['lda_mean'][components[0]] - population['lda_std'][components[0]], y0=population['lda_mean'][components[1]] - population['lda_std'][components[1]],
        x1=population['lda_mean'][components[0]] + population['lda_std'][components[0]], y1=population['lda_mean'][components[1]] + population['lda_std'][components[1]],
    )

fig

In [ ]:
X = np.concatenate([populations[name]['data'] for name in populations])
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)

# 1. Normalizando
Z = X - mean # As colunas devem ter média 0
Z = Z / std # As colunas devem ter desvio padrao 1 (em alguns problemas, a variancia dos dados pode ser utilizada como fonte de informação para a classificação, nesse caso não normalize o desvio padrao). 
            # Ao comentar essa linha, o resultado é identico ao do sklearn
Z = np.matrix(Z)

# print(Z)
# 2. Matriz de covariancia
covariance = Z.T * Z

# 3. Decomposição

eig_values, eig_vectors = np.linalg.eig(covariance)

D, P = np.diag(eig_values), eig_vectors

# 4. Ordenação decrescende dos autovalores
eig_desc = np.flip(np.argsort(eig_values))
P = P[:, eig_desc] # A matriz P é o operandor linear que nos interessa
D = D[:, eig_desc]


# 5. Efetuando a transformação linear
Z_ = Z * P

In [ ]:
fig = go.Figure(layout=dict(title='Resultado do PCA implementado manualmente', height=800))

for name in populations:
    population = populations[name]
    mask = all_labels == name
    fig.add_trace(
        go.Scatter3d(
            x = np.array(Z_)[mask, 0],
            y = np.array(Z_)[mask, 1],
            z = np.array(Z_)[mask, 2],
            name=name,
            mode = 'markers',
            marker=dict(
                size=3
            )
        )
    )
fig.show()


fig = go.Figure(layout=dict(title='As duas componentes mais relevantes do PCA implementado manualmente', height=800))

for name in populations:
    population = populations[name]
    mask = all_labels == name
    fig.add_trace(
        go.Scatter(
            x = np.array(Z_)[mask, 0],
            y = np.array(Z_)[mask, 1],
            name=name,
            mode = 'markers',
        )
    )
fig.show()

Z_pca = pca.transform(X)

fig = go.Figure(layout=dict(title='As duas componentes mais relevantes do PCA segundo o sklearn', height=800))
for name in populations:
    population = populations[name]
    mask = all_labels == name
    fig.add_trace(
        go.Scatter(
            x = Z_pca[mask, 0],
            y = Z_pca[mask, 1],
            name=name,
            mode = 'markers',
        )
    )
fig.show()

In [ ]:
full_space = np.array([
    list([x, y, z])
    for x in np.linspace(-20, 20, 20)
    for y in np.linspace(-20, 20, 20)
    for z in np.linspace(-20, 20, 20)
])

full_lda = lda.transform(full_space)
full_labels = lda.predict(full_space)

fig = go.Figure(layout=dict(title='Divisão do espaço conforme o classificador LDA', height=800))

for name in ['a', 'b', 'b\'']:
    population = populations[name]
    mask = full_labels == name
    fig.add_trace(
        go.Scatter3d(
            x = full_space[mask, 0],
            y = full_space[mask, 1],
            z = full_space[mask, 2],
            name=name,
            mode = 'markers',
            marker=dict(
                size=2
            )
        )
    )
fig.show()

fig = go.Figure(layout=dict(title='Divisão do espaço TRANSFORMADO conforme o classificador LDA', height=800))

for name in ['a', 'b', 'b\'']:
    population = populations[name]
    mask = full_labels == name
    fig.add_trace(
        go.Scatter(
            x = full_lda[mask, 0],
            y = full_lda[mask, 1],
            name=name,
            mode = 'markers',
            marker=dict(
            )
        )
    )
fig.show()

fig = go.Figure(layout=dict(title='LDA', height=800))
for name in ['a', 'b', 'b\'']:
    population = populations[name]
    components = [0, 1]
    x = population['lda'][:,components[0]]
    y = population['lda'][:,components[1]]
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode = 'markers',
            name=name,
                
        )
    )
fig.show()